In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from tensorflow.keras import layers, models

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train[..., None] / 255.0
x_test = x_test[..., None] / 255.0
input_shape = x_train.shape[1:]
latent_dim = 64

In [ ]:
def show_images(images, title):
    plt.figure(figsize=(10, 2))
    for i, img in enumerate(images):
        plt.subplot(1, 5, i + 1)
        plt.imshow(img.squeeze(), cmap='gray')
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

In [ ]:
def sample_noise(size):
    return np.random.normal(5.0, 1.0, size)

In [ ]:
# Encoder
input_layer = layers.Input(shape=input_shape)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
latent = layers.Dense(latent_dim, activation='relu')(x)
encoder = models.Model(input_layer, latent)

# Decoder
decoder_input = layers.Input(shape=(latent_dim,))
x = layers.Dense(128, activation='relu')(decoder_input)
x = layers.Dense(7*7*64, activation='relu')(x)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, (3, 3), strides=2, activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation='relu', padding='same')(x)
output_layer = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
decoder = models.Model(decoder_input, output_layer)